<span style="color: #9370DB">*Daniela Jiménez*</span>

<span style="color: #9370DB">*Bárbara Flores*</span>

# Estimating Gender Discrimination in the Workplace

In this exercise we'll use data from the 2018 US Current Population Survey (CPS) to try and estimate the effect of being a woman on workplace compensation. 

Note that our focus will be *only* on differential compensation in the work place, and as a result it is important to bear in mind that our estimates are not estimates of *all* forms of gender discrimination. For example, these analyses will not account for things like gender discrimination in terms of *getting* jobs. We'll discuss this in more detail below.

## Exercise 1: 

Begin by downloading and importing 2018 CPS data from [https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey](https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey). The file is called `morg18.dta` and is a Stata dataset. Additional data on the dataset can be found by following the links in the README.txt file in the folder, but for the moment it is sufficient to know this is a national survey run in the United States.

The survey does include some survey weights we won't be using (i.e. not everyone in the sample was included with the same probability), so the numbers we estimate will not be perfect estimates of the gender wage gap in the United States, but they are pretty close.

In [1]:
import pandas as pd

path = "https://github.com/nickeubank/MIDS_Data/raw/master/Current_Population_Survey/morg18.dta"
morg18 = pd.read_stata(path)
morg18.head()

## Exercise 2:

Because our interest is only in-the-workplace wage discrimination among full-time workers, we need to start by subsetting our data for people currently employed (and "at work", not "absent") at the time of this survey using the `lfsr94` variable, who are employed full time (meaning that their usual hours per week—`uhourse`—is 35 or above).

As noted above, this analysis will miss many forms of gender discrimination. For example, in dropping anyone who isn't working, we immediately lose any women who couldn't get jobs, or who chose to lose the workforce because the wages they were offered (which were likely lower than those offered men) were lower than they were willing / could accept. And in focusing on full time employees, we miss the fact women may not be offered full time jobs at the same rate as men. 

In [ ]:
morg18 = morg18[morg18["lfsr94"] == "Employed-At Work"]
morg18 = morg18[morg18["uhourse"] >= 35]

## Exercise 3

Now let's estimate the basic wage gap for the United States!

Earnings per week worked can be found in the `earnwke` variable. Using the variable `sex` (1=Male, 2=Female), estimate the gender wage gap in terms of wages per hour worked!

(You may also find it helpful, for context, to estimate the average hourly pay by dividing weekly pay by `uhourse`.)

In [ ]:
average_salary_male_weekly = morg18[morg18["sex"] == 1]["earnwke"].mean()
average_salary_female_weekly = morg18[morg18["sex"] == 2]["earnwke"].mean()

weekly_gender_wage_gap = average_salary_male_weekly - average_salary_female_weekly

print(
    f"\nThe average weekly salary for men is ${average_salary_male_weekly:,.2f}")
print(
    f"The average weekly salary for women is ${average_salary_female_weekly:.2f}")
print(
    f"\nThe gender wage gap in the United States is ${weekly_gender_wage_gap:.2f} per week"
)


The average weekly salary for men is $1,204.73
The average weekly salary for women is $985.68

The gender wage gap in the United States is $219.05 per week


In [ ]:
salary_per_hour_male = (
    morg18[morg18["sex"] == 1]["earnwke"] /
    morg18[morg18["sex"] == 1]["uhourse"]
).mean()
salary_per_hour_female = (
    morg18[morg18["sex"] == 2]["earnwke"] /
    morg18[morg18["sex"] == 2]["uhourse"]
).mean()

hourly_gender_wage_gap = salary_per_hour_male - salary_per_hour_female

print(f"The average hourly salary for men is ${salary_per_hour_male:,.2f}")
print(f"The average hourly salary for women is ${salary_per_hour_female:,.2f}")
print(
    f"\nThe gender wage gap in the United States is ${hourly_gender_wage_gap:,.2f} per hour"
)

The average hourly salary for men is $27.88
The average hourly salary for women is $23.80

The gender wage gap in the United States is $4.08 per hour


## Exercise 4

Assuming 48 work weeks in a year, calculate annual earnings for men and women. Report the difference in dollars and in percentage terms.

In [ ]:
annual_earnings_male = average_salary_male_weekly * 48
annual_earnings_female = average_salary_female_weekly * 48

earnings_difference = annual_earnings_male - annual_earnings_female

percentage_difference = (earnings_difference / annual_earnings_female) * 100

print(f"Annual earnings for men: ${annual_earnings_male:,.0f}")
print(f"Annual earnings for women: ${annual_earnings_female:,.0f}")
print(
    f"The difference in earnings is ${earnings_difference:,.0f} which represents {percentage_difference:.2f}% more earnings for men compared to women."
)

Annual earnings for men: $57,827
Annual earnings for women: $47,313
The difference in earnings is $10,514 which represents 22.22% more earnings for men compared to women.


## Exercise 5

We just compared all full-time working men to all full-time working women. For this to be an accurate *causal* estimate of the effect of being a woman in the work place, what must be true of these two groups? What is one reason that this may *not* be true?

>For this comparison to be accurate, both groups must be similar in other income-affecting characteristics such as age, race, educational level, etc. If these conditions were similar for both groups or had similar distributions, comparing the means of the two groups could provide a precise estimate of the effect of being a woman in the workplace.
>
>One reason why this assumption might not hold true is if women who have full-time jobs tend to have a higher average level of education than men (since they are more likely to have full-time jobs). In such a case, it would be necessary to control for educational level in our comparison to obtain a precise estimate of the effect of being a woman in the workplace.

## Exercise 6

One answer to the second part of Exercise 5 is that working women are likely to be younger, since a larger portion of younger women are entering the workforce as compared to older generations.

To *control* for this difference, let's now regress annual earnings on gender, age, and age-squared (the relationship between age and income is generally non-linear). What is the implied average annual wage difference between women and men? Is it different from your raw estimate? 

## Exercise 7

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we're basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

## Exercise 8

Now let's add to our regression an indicator variable for whether the respondent has at least graduated high school, and an indicator for whether the respondent at least has a BA. 

In answering this question, use the following table of codes for the variable `grade92`. 

Education is coded as follows:
    
![CPS Educ Codes](../images/cps_educ_codes.png)

## Exercise 9 

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we are once more basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

## Exercise 10

Given how the coefficient on `female` has changed between Exercise 6 and Exercise 8, what can you infer about the educational attainment of the women in your survey data (as compared to the educational attainment of men)?

## Exercise 11

What does that tell you about the *potential outcomes* of men and women before you added education as a control?

## Exercise 12

Finally, let's include *fixed effects* for the type of job held by each respondent. 

Fixed effects are a method used when we have a nested data structure in which respondents belong to groups, and those groups may all be subject to different pressures. In this context, for example, we can add fixed effects for the industry of each respondent—since wages often vary across industries, controlling for industry is likely to improve our estimates. Use `ind02` to control for industry.

(Note that fixed effects are very similar in principle to hierarchical models. There are some differences [you will read about](../fixed_effects_v_hierarchical.ipynb) for our next class, but they are designed to serve the same role, just with slightly different mechanics). 

When we add fixed effects for groups like this, our interpretation of the other coefficients changes. Whereas in previous exercises we were trying to explain variation in men and women's wages *across all respondents*, we are now effectively comparing men and women's wages *within each employment sector*. Our coefficient on `female`, in other words, now tells us how much less (on average) we would expect a woman to be paid than a man *within the same industry*, not across all respondents. 

(Note that running this regression will result in lots of coefficients popping up you don't care about. We'll introduce some more efficient methods for adding fixed effects that aren't so messy in a later class -- for now, you can ignore those coefficients!)

## Exercise 13

Now that we've added industry fixed effects, what groups are we implicitly treated as counter-factuals for one another now? 

## Exercise 14

What happened to your estimate of the gender wage gap when you added industry fixed effects? What does that tell you about the industries chosen by women as opposed to men?

When you're done, please come read [this discussion](discussion_regressions_incomeineq.ipynb).